# Direct diagonalization of tight-binding sd-model with on-site Coulomb repulsion.

In [2]:
#math modules
import math
import numpy as np

import time

In [3]:
# Kinetic energy
n_site = 2
n_states = math.pow(2, n_site)



In [24]:
# s site system, 2 electrons, NO periodic boundary conditions
from numpy import linalg as LA
U_c = 10
t = -1
ham = [[-U_c, t, t, 0], [t, U_c, 0, t], [t, 0, U_c, t], [0, t, t, -U_c]]
eigenvalues, eigenvectors = LA.eig(np.array(ham))

#eigen-vectors are columns (not rows)

In [25]:
eigenvalues

array([-10.19803903, -10.        ,  10.19803903,  10.        ])

In [21]:
math.sqrt(U_c * U_c + 4 * t * t)

10.198039027185569

In [17]:
eigenvectors

array([[-7.03665523e-01, -7.07106781e-01, -6.96766179e-02,
         8.98491639e-16],
       [-6.96766179e-02,  5.34087030e-16,  7.03665523e-01,
        -7.07106781e-01],
       [-6.96766179e-02,  7.53649044e-16,  7.03665523e-01,
         7.07106781e-01],
       [-7.03665523e-01,  7.07106781e-01, -6.96766179e-02,
         9.42900560e-16]])

In [45]:
import itertools
'''
def combinations(iterable, r):
  pool = tuple(iterable)
  n = len(pool)
  if r > n:
    return
  indices = list(range(r))
  yield tuple(pool[i] for i in indices)
  while True:
    for i in reversed(range(r)):
      if indices[i] != i + n - r:
        break
    else:
      return
    indices[i] += 1
    for j in range(i+1, r):
      indices[j] = indices[j-1] + 1
    yield tuple(pool[i] for i in indices)

iterable = [1, 2, 3]
r = 3
result = list(combinations(iterable, r))
print(result)
'''

'\ndef combinations(iterable, r):\n  pool = tuple(iterable)\n  n = len(pool)\n  if r > n:\n    return\n  indices = list(range(r))\n  yield tuple(pool[i] for i in indices)\n  while True:\n    for i in reversed(range(r)):\n      if indices[i] != i + n - r:\n        break\n    else:\n      return\n    indices[i] += 1\n    for j in range(i+1, r):\n      indices[j] = indices[j-1] + 1\n    yield tuple(pool[i] for i in indices)\n\niterable = [1, 2, 3]\nr = 3\nresult = list(combinations(iterable, r))\nprint(result)\n'

In [108]:
aa=list(itertools.combinations('0123', 3))

In [109]:
aa

[('0', '1', '2'), ('0', '1', '3'), ('0', '2', '3'), ('1', '2', '3')]

In [110]:
len(aa)

4

In [111]:
def convert_to_spin(list_of_comb, n_nodes, n_el):
    list_f = []
    for i in range(len(aa)):
        state = ''
        for j in range(n_el):
            nn = int(list_of_comb[i][j])
            
            if nn >= n_nodes:
                nn_1 = nn - n_nodes
                state = state + str(nn_1)
                state = state + 'd_'
            else:
                state = state + str(nn)
                state = state + 'u_'
        list_f.append(state)
    return list_f


In [112]:
zz = convert_to_spin(aa, 3, 2)

In [113]:
zz

['0u_1u_', '0u_1u_', '0u_2u_', '1u_2u_']

In [152]:
def kinetic_matrix_element(state_1, state_2, n_el, n_nodes, t):
    jumps = 0
    for i in range(n_el):
        n_1 = int(state_1[i])
        n_2 = int(state_2[i])
        if n_1 == n_2:
            jumps = jumps
        elif n_1 == 0 and n_2 == n_nodes - 1:
            jumps = jumps + 1
        elif n_1 == n_nodes - 1 and n_2 == 0:
            jumps = jumps + 1
        elif n_1 == n_nodes and n_2 == 2 * n_nodes - 1:
            jumps = jumps + 1
        elif n_1 == 2 * n_nodes - 1 and n_2 == n_nodes:
            jumps = jumps + 1    
        elif (n_1 == n_2 + 1 and n_1 != n_nodes) or (n_1 == n_2 - 1 and n_1 != n_nodes - 1):
            jumps = jumps + 1
        else:
            jumps = jumps + 2
    me = 0
    if jumps == 1:
        me = t
    return me
           

In [153]:
aa

[('0', '1'), ('0', '2'), ('0', '3'), ('1', '2'), ('1', '3'), ('2', '3')]

In [155]:
convert_to_spin(aa, 2, 2)

['0u_1u_', '0u_0d_', '0u_1d_', '1u_0d_', '1u_1d_', '0d_1d_']

In [157]:
kinetic_matrix_element(aa[0], aa[3], 2, 2, 1)

0

In [166]:
def Coulomb_matrix_element(state_1, state_2, n_el, n_nodes, U_c):
    me = 0
    if state_1 == state_2:
        st = [-1 ] * n_nodes * 2
        for i in range(n_el):
            st[int(state_1[i])] = 1
        #print(st)
        for i in range(n_nodes):
            me = me + U_c / 2 * st[i] * st[i + n_nodes]
    else:
        me = 0
    return me

In [167]:
Coulomb_matrix_element(aa[1], aa[1], 2, 2, 1)

1.0

In [168]:
def kin_en_matr(states, n_el, n_nodes, t):
    
    m_size = len(states)
    k_m = np.zeros((m_size, m_size))
    for i in range(m_size):
        for j in range (m_size):
            k_m[i][j] = kinetic_matrix_element(states[i], states[j], n_el, n_nodes, t)
    return k_m

def Coul_en_matr(states, n_el, n_nodes, t):
    m_size = len(states)
    k_m = np.zeros((m_size, m_size))
    for i in range(m_size):
        for j in range (m_size):
            k_m[i][j] = Coulomb_matrix_element(states[i], states[j], n_el, n_nodes, t)
    return k_m

def ham_tot(states, n_el, n_nodes, t, U_c):
    return kin_en_matr(states, n_el, n_nodes, t) + Coul_en_matr(states, n_el, n_nodes, U_c)

In [169]:
#mm = kin_en_matr(aa, 3, 2, 2) + Coul_en_matr(aa, 3, 2, 3)
#mm

In [187]:
n_el = 4
n_nodes = 3
t = 1
U_c = 10
ll = ''
for i in range(n_nodes * 2):
    ll = ll + str(i)
print(ll)
aa=list(itertools.combinations(ll, n_el))
print(aa)
ham = kin_en_matr(aa, n_el, n_nodes, t)
ham = ham_tot(aa, n_el, n_nodes, t, U_c)
#print(ham)

012345
[('0', '1', '2', '3'), ('0', '1', '2', '4'), ('0', '1', '2', '5'), ('0', '1', '3', '4'), ('0', '1', '3', '5'), ('0', '1', '4', '5'), ('0', '2', '3', '4'), ('0', '2', '3', '5'), ('0', '2', '4', '5'), ('0', '3', '4', '5'), ('1', '2', '3', '4'), ('1', '2', '3', '5'), ('1', '2', '4', '5'), ('1', '3', '4', '5'), ('2', '3', '4', '5')]


In [188]:
eigenvalues, eigenvectors = LA.eig(np.array(ham))

In [189]:
eigenvalues

array([-6.        , -3.        , -6.        , -6.56018773, -5.09950494,
       -3.73686381, 15.29705154, 15.        , 15.09950494, -6.41421356,
       -5.        , -3.58578644, -6.        , -3.        , -6.        ])

In [191]:
eig = np.transpose(eigenvectors)
eig[3]

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  3.42439736e-02,
       -3.69153249e-01,  4.73216449e-01, -3.69153249e-01,  6.84879471e-02,
       -3.69153249e-01,  4.84792263e-34,  4.73216449e-01, -3.69153249e-01,
        3.42439736e-02,  0.00000000e+00,  0.00000000e+00])

In [199]:
def show_state(states, state):
    state_str = ''
    ll = len(state)
    for i in range(ll):
        if state[i] > 1 / ll:
            #ss = convert_to_spin(states[i], n_nodes, n_el)
            #{price:.2f}
            sss = f"{state[i]:.2f}"
            state_str = state_str + sss + ' * ' + states[i] + ' + '
    return state_str
show_state(convert_to_spin(aa, n_nodes, n_el), eig[3])
    

'0.47 * 0u_1u_1d_2d_ + 0.07 * 0u_2u_0d_2d_ + 0.47 * 1u_2u_0d_1d_ + '